#### Imports

In [26]:
# if package not installed, install with command

#!pip install torchxrayvision

In [27]:
import torchxrayvision as xrv

import skimage
#Image Processing for Python
#scikit-image (a.k.a. skimage) is a collection of algorithms for image processing and computer vision

import cv2

import torch
#The torch package contains data structures for multi-dimensional tensors and defines mathematical operations over these tensors.

import torch.nn.functional as F

import pandas as pd

#### Import a chest xray image

In [28]:
img = skimage.io.imread('/content/02aa804e-bde0afdd-112c0b34-7bc16630-4e384014.jpg')

#### Display image details
*the image is read as an array (type = numpy.ndarray)*

In [29]:
img

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

#### Select model to use for feature extraction
*we choose densenet121-res224-chex as the model weights name from Stanford university CheXpert models*

In [30]:
model_weights_name = "densenet121-res224-chex"

# Select model 
model = xrv.models.DenseNet(weights = model_weights_name)

#Initialize the densefeature_embeddings with an empty list:
densefeature_embeddings = []

#Initialize the prediction_embeddings with an empty list:
prediction_embeddings = []

#Scales images to be roughly [-1024 1024]:
img = xrv.datasets.normalize(img, 255)


In [31]:
model

XRV-DenseNet121-densenet121-res224-chex

In [32]:
# For each image check if they are 2D arrays
if len(img.shape) > 2:
    img = img[:, :, 0]
if len(img.shape) < 2:
    print("Error: Dimension lower than 2 for image!")

In [33]:
# Add color channel for prediction
#Resize using OpenCV
img = cv2.resize(img, (224, 224), interpolation = cv2.INTER_AREA)   
img = img[None, :, :]

In [34]:
img

array([[[-1024.0001, -1024.0001, -1024.0001, ..., -1024.0001,
         -1024.0001, -1024.0001],
        [-1024.0001, -1024.0001, -1024.0001, ..., -1024.0001,
         -1024.0001, -1024.0001],
        [-1024.    , -1024.    , -1024.    , ..., -1024.    ,
         -1024.    , -1024.    ],
        ...,
        [-1024.0001, -1024.0001, -1024.0001, ..., -1024.0001,
         -1024.0001, -1024.0001],
        [-1024.0001, -1024.0001, -1024.0001, ..., -1024.0001,
         -1024.0001, -1024.0001],
        [-1024.0001, -1024.0001, -1024.0001, ..., -1024.0001,
         -1024.0001, -1024.0001]]], dtype=float32)

In [35]:
# Select if you want to use CUDA support for GPU (optional as it is usually pretty fast even in CPUT)
cuda = False

In [36]:
output = {}
with torch.no_grad():
    img = torch.from_numpy(img).unsqueeze(0)
    if cuda:
        img = img.cuda()
        model = model.cuda()
      
    # Extract dense features
    feats = model.features(img)
    feats = F.relu(feats, inplace=True)
    feats = F.adaptive_avg_pool2d(feats, (1, 1))
    densefeatures = feats.cpu().detach().numpy().reshape(-1)
    densefeature_embeddings = densefeatures

    # Extract predicted probabilities of considered 18 classes:
    # ['Atelectasis','Consolidation','Infiltration','Pneumothorax','Edema','Emphysema',Fibrosis',
    #  'Effusion','Pneumonia','Pleural_Thickening','Cardiomegaly','Nodule',Mass','Hernia',
    #  'Lung Lesion','Fracture','Lung Opacity','Enlarged Cardiomediastinum']
    preds = model(img).cpu()
    predictions = preds[0].detach().numpy()
    prediction_embeddings = predictions  

# Return embeddings
#return densefeature_embeddings, prediction_embeddings

In [37]:
print(densefeature_embeddings)
len(densefeature_embeddings)

[0.         0.         0.1838883  ... 0.00537282 0.         0.10955384]


1024

In [38]:
print(preds)
len(preds)


tensor([[0.2119, 0.1532, 0.5000, 0.0455, 0.2962, 0.5000, 0.5000, 0.0979, 0.1885,
         0.5000, 0.1239, 0.5000, 0.5000, 0.5000, 0.2773, 0.4033, 0.3089, 0.1374]])


1

In [39]:
print(predictions)
len(predictions)

[0.21186149 0.15321094 0.5        0.04549948 0.29623356 0.5
 0.5        0.09788145 0.1885436  0.5        0.12390422 0.5
 0.5        0.5        0.2773131  0.4033455  0.3089283  0.13744338]


18

In [40]:
print(prediction_embeddings)
len(prediction_embeddings)

[0.21186149 0.15321094 0.5        0.04549948 0.29623356 0.5
 0.5        0.09788145 0.1885436  0.5        0.12390422 0.5
 0.5        0.5        0.2773131  0.4033455  0.3089283  0.13744338]


18

### Creating the dataframe from the arrays:

##### concatenation:

In [41]:
 df_vision_dense_embeddings_fusion = pd.DataFrame(densefeature_embeddings.reshape(1,-1), columns=['vd_'+str(i) for i in range(densefeature_embeddings.shape[0])])
 

In [42]:
df_vision_dense_embeddings_fusion

,vd_0,vd_1,vd_2,vd_3,vd_4,vd_5,vd_6,vd_7,vd_8,vd_9,...,vd_1014,vd_1015,vd_1016,vd_1017,vd_1018,vd_1019,vd_1020,vd_1021,vd_1022,vd_1023
0,0.0,0.0,0.183888,0.007851,0.024935,0.012488,0.153937,0.021945,0.0,0.018086,...,0.0,0.011642,0.0,0.019547,0.050666,0.0,0.004459,0.005373,0.0,0.109554


In [43]:
 df_vision_predictions_embeddings_fusion = pd.DataFrame(prediction_embeddings.reshape(1,-1), columns=['vp_'+str(i) for i in range(prediction_embeddings.shape[0])])
 

In [44]:
df_vision_predictions_embeddings_fusion

,vp_0,vp_1,vp_2,vp_3,vp_4,vp_5,vp_6,vp_7,vp_8,vp_9,vp_10,vp_11,vp_12,vp_13,vp_14,vp_15,vp_16,vp_17
0,0.211861,0.153211,0.5,0.045499,0.296234,0.5,0.5,0.097881,0.188544,0.5,0.123904,0.5,0.5,0.5,0.277313,0.403345,0.308928,0.137443
